# IMPORT LIBRARIES

In [2]:
import pandas as pd
import numpy as np
import re
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from pandas.io.json import json_normalize
import requests
from bs4 import BeautifulSoup

# ACCESS SPOTIFY API

In [2]:
cid ="53af5d19b4f54767af1e9c59744bbd30" 
secret = "e7c9ab58bf324c5d95e8b8e2ff6c7336"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# QUERY - Top Cities Who Listen to Artist

In [3]:
enter_artist = 'yoko ono'

results = sp.search(q='artist:' + enter_artist, type='artist')
if results['artists']['items'] == []:
    print('no match')
else:
    two_dict = results['artists']['items'][0]['external_urls']['spotify']
    about = two_dict + "/about"
print("Link to Artist Biography: " + about)

html = requests.get(about).content
soup = BeautifulSoup(html, "lxml")
soup.ul.li.span

city = [x.text.strip() for x in soup.find_all('span', attrs = {"class" : "horizontal-list__item__title"})]
city

Link to Artist Biography: https://open.spotify.com/artist/2s4tjL6W3qrblOe0raIzwJ/about


['Chicago, US',
 'Mexico City, MX',
 'New York City, US',
 'Los Angeles, US',
 'Stockholm, SE']

# QUERY - Monthly Listeners of Artist

In [4]:
listeners = (soup.find_all('h3', attrs = {"class" : "insights__column__number"})[0]).text.strip()
listeners

'1,074,357'

# QUERY - Followers of Artist

In [5]:
followers = (soup.find_all('h3', attrs = {"class" : "insights__column__number"})[1]).text.strip()
followers

'58,558'

# ...TEST LIST

In [6]:
top_artists = ['the beatles', 'dj shadow', 'david bowie', 'the cure', 'nina simone']

# LOOP - Cities

In [7]:
def get_cities(enter_artist):
    results = sp.search(q='artist:' + enter_artist, type='artist')
    if results['artists']['items'] == []:
        return 'NO MATCH'
    else:
        two_dict = results['artists']['items'][0]['external_urls']['spotify']
        about = two_dict + "/about"
        html = requests.get(about).content
        soup = BeautifulSoup(html, "lxml")
        city = [x.text.strip() for x in soup.find_all('span', attrs = {"class" : "horizontal-list__item__title"})]
        return city

print(list(map(get_cities, top_artists)))

[['Mexico City, MX', 'São Paulo, BR', 'Santiago, CL', 'Chicago, US', 'Los Angeles, US'], ['Mexico City, MX', 'Los Angeles, US', 'Chicago, US', 'New York City, US', 'London, GB'], ['Mexico City, MX', 'São Paulo, BR', 'Santiago, CL', 'Paris, FR', 'Los Angeles, US'], ['Mexico City, MX', 'Santiago, CL', 'São Paulo, BR', 'Buenos Aires, AR', 'Los Angeles, US'], ['Paris, FR', 'São Paulo, BR', 'Mexico City, MX', 'London, GB', 'Madrid, ES']]


# LOOP - Listeners

In [8]:
def get_listeners(enter_artist):
    results = sp.search(q='artist:' + enter_artist, type='artist')
    if results['artists']['items'] == []:
        return 'NO MATCH'
    else:
        two_dict = results['artists']['items'][0]['external_urls']['spotify']
        about = two_dict + "/about"
        html = requests.get(about).content
        soup = BeautifulSoup(html, "lxml")
    listeners = (soup.find_all('h3', attrs = {"class" : "insights__column__number"})[0]).text.strip()
    return listeners

print(list(map(get_listeners, top_artists)))

['20,558,219', '1,524,103', '12,609,061', '8,771,482', '4,665,980']


# LOOP - Followers

In [9]:
def get_followers(enter_artist):
    results = sp.search(q='artist:' + enter_artist, type='artist')
    if results['artists']['items'] == []:
        return 'NO MATCH'
    else:
        two_dict = results['artists']['items'][0]['external_urls']['spotify']
        about = two_dict + "/about"
        html = requests.get(about).content
        soup = BeautifulSoup(html, "lxml")
    followers = (soup.find_all('h3', attrs = {"class" : "insights__column__number"})[1]).text.strip()
    return followers

print(list(map(get_followers, top_artists)))

['14,003,775', '343,945', '4,664,781', '2,227,529', '1,475,175']


# ------------------------------------------------------------------------------------------------------------

# PITCHFORK DATA

In [3]:
pitchfork = pd.read_csv('pitchfork.csv')
pitchfork.sort_values('score', ascending = False)

,id,artist,content,genre,label,album,score,best_new_music,author,pub_date,year
7325,15957,talk talk,There are many ways for a band to follow up a ...,rock,ba da bing!,laughing stock,10.0,1,jess harvell,2011-10-21,2011
2419,20483,the rolling stones,"The story of the Baby Boomers, and their movem...",rock,rolling stones,sticky fingers,10.0,1,mark richardson,2015-06-19,1971
9647,14288,the cure,The late 1980s and early 90s were the Cure's h...,rock,rhino,disintegration [deluxe edition],10.0,1,nitsuh abebe,2010-06-10,2010
4008,19240,mobb deep,"The foreboding, faraway skree announcing Mobb ...",rap,self-released,the infamous,10.0,1,jayson greene,2014-04-09,2014
4007,19240,mobb deep,"The foreboding, faraway skree announcing Mobb ...",rap,self-released,the infamous,10.0,1,jayson greene,2014-04-09,1995
...,...,...,...,...,...,...,...,...,...,...,...
15772,9464,jet,NaN,electronic,elektra,shine on,0.0,0,ray suzuki,2006-10-02,2006
20859,6255,liz phair,It could be said that Liz Phair's greatest ass...,rock,capitol,liz phair,0.0,0,matt lemay,2003-06-24,2003
15771,9464,jet,NaN,rock,elektra,shine on,0.0,0,ray suzuki,2006-10-02,2006
18985,5607,travis morrison,Travis Morrison got his ass kicked. He tells ...,pop/r&b,barsuk,travistan,0.0,0,chris dahlen,2004-09-27,2004


# GENRE vs. SCORE

In [ ]:
genre_score = pitchfork[["genre","score","best_new_music"]]
genre_score_agg = ((genre_score.groupby('genre')).agg(['mean'])).sort_values(('best_new_music', 'mean'), ascending=False)
genre_score_agg

In [13]:
pitchfork['best_new_music'].sum()

in_best = pitchfork[pitchfork["best_new_music"] == 1]
artists_in_best = in_best[["artist","best_new_music"]]
artists_best_count = (artists_in_best.groupby('artist').agg([sum])).sort_values(('best_new_music', 'sum'), ascending = False)
over_5 = artists_best_count[artists_best_count[("best_new_music","sum")] >= 5]
artist_list = [x for x in over_5.index]
artist_list

['m83',
 'kanye west',
 'lcd soundsystem',
 'saint etienne',
 'm.i.a.',
 'spiritualized',
 'animal collective',
 'the knife',
 'the velvet underground',
 'r.e.m.',
 'beach house',
 'jawbreaker',
 'how to dress well',
 'john lennon',
 'ride',
 'menomena',
 'bikini kill',
 'yoko ono',
 'beastie boys',
 'burial',
 'miles davis',
 'oneohtrix point never',
 'the smashing pumpkins',
 'drake',
 'kendrick lamar',
 'deerhunter',
 'caribou',
 'janelle mone',
 'sufjan stevens',
 'my bloody valentine',
 'memory tapes',
 'jay z',
 'robyn',
 'fleetwood mac',
 'the unicorns',
 'radiohead',
 'the antlers',
 'cut copy',
 'liars',
 'sunset rubdown',
 'art of noise',
 'david bowie',
 'grandaddy',
 'throbbing gristle',
 'deerhoof',
 'the decemberists',
 'nick cave & the bad seeds']

In [ ]:
list(map(get_listeners, artist_list))

In [9]:
genre_best_new = pitchfork[["genre","score","best_new_music","year"]]
genre_best_new.head()

,genre,score,best_new_music,year
0,electronic,9.3,0,1998
1,metal,7.9,0,2016
2,rock,7.3,0,2016
3,rock,7.3,0,2016
4,rock,9.0,1,2016


In [ ]:
genre_score_agg = ((genre_score.groupby('genre')).agg(['mean'])).sort_values(('best_new_music', 'mean'), ascending=False)
genre_score_agg